In [65]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import learning_curve
import matplotlib.pyplot as plt

In [66]:
df = pd.read_csv('df_RIGHTTARGET')
df = df.drop(df.columns[0], axis=1)
df.head()

C:\Users\HP\AppData\Local\Temp\ipykernel_19372\2929177649.py:1: DtypeWarning: Columns (19,20,21,22,23) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('df_RIGHTTARGET')


,TimeInt,TimeStr,Act_Receptnummer,Active_nummer_wisselblok,Pauze,StangData.Correctie_Totale_lengte,StangData.Correctie_Totale_lengte_operator_trek1en3,StangData.Offset_knipklem_knippositie_Operator,StangData.Pos-UitloCor_2_trek24,StangData.Pos_inloCor_1_trek13,...,stangbuff_0,stangbuff_1,stangkwijt_buf,stangkwijt_timer,totaal_aantal,source_file,d_knip,d_goed,d_total,bad_spaak
0,1761290871,24/10/2025 07:27:51,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv,0.0,0.0,0.0,0
1,1761290876,24/10/2025 07:27:56,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv,0.0,0.0,0.0,0
2,1761290881,24/10/2025 07:28:01,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv,0.0,0.0,0.0,0
3,1761290886,24/10/2025 07:28:06,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv,0.0,0.0,0.0,0
4,1761290891,24/10/2025 07:28:11,74,10,0,-0.2,0.0,-1.0,-0.9,1.1,...,0,0,0,0,15512946,20251024 SVRM4-HisLog18.csv,0.0,0.0,0.0,0


In [67]:
df = df.sort_values("TimeInt").reset_index(drop=True)

In [68]:
df["is_bad"] = (df["bad_spaak"] > 0).astype(int)


# Kernfeatures
top_features = [
    "bufferlevel",
    "stangkwijt_timer",
    "stangbuff_0",
    "stangbuff_1",
    "StangData.Correctie_Totale_lengte_operator_trek1en3",
    "StangData.Offset_knipklem_knippositie_Operator",
    "knipteller",
    "Act_Receptnummer",
    "totaal_aantal",
    "stangbuf_2"
]

# Sensor/buffer kolommen waarvoor we lags en rolling features willen
core_sensors = [
    "bufferlevel",
    "stangkwijt_timer",
    "stangbuff_0",
]

for col in core_sensors:
    # lags
    for lag in [1, 2, 3]:
        df[f"{col}_lag{lag}"] = df[col].shift(lag)

    # rolling windows
    for w in [10, 20, 50]:
        df[f"{col}_roll_mean{w}"] = df[col].rolling(w).mean()


df = df.fillna(0)
X_features = [
    # originele sensoren
    "bufferlevel",
    "stangkwijt_timer",
    "stangbuff_0",

    # lags
    "bufferlevel_lag1", "bufferlevel_lag2", "bufferlevel_lag3",
    "stangkwijt_timer_lag1", "stangkwijt_timer_lag2", "stangkwijt_timer_lag3",
    "stangbuff_0_lag1", "stangbuff_0_lag2", "stangbuff_0_lag3",

    # rolling windows
    "bufferlevel_roll_mean10", "bufferlevel_roll_mean20", "bufferlevel_roll_mean50",
    "stangkwijt_timer_roll_mean10", "stangkwijt_timer_roll_mean20", "stangkwijt_timer_roll_mean50",
    "stangbuff_0_roll_mean10", "stangbuff_0_roll_mean20", "stangbuff_0_roll_mean50",
]


# De kolommen voor het model: top_features + alle lag/rolling features
X_features = top_features + [
    c for c in df.columns
    if any(c.startswith(col) and ("lag" in c or "roll" in c) for col in core_sensors)
]

X = df[X_features]
y = df["is_bad"]


In [69]:
train_size = int(0.8 * len(df))
X_train = X.iloc[:train_size]
y_train = y.iloc[:train_size]
X_test  = X.iloc[train_size:]
y_test  = y.iloc[train_size:]

In [70]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix

In [59]:
rf = RandomForestClassifier(class_weight="balanced", random_state=42, n_jobs=-1)

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [8, 12, 16, None],
    'min_samples_leaf': [5, 10, 20]
}

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring='f1',
    verbose=2,
    n_jobs
grid_search.fit(X_train, y_train)

print("Beste parameters:", grid_search.best_params_)
print("Beste f1-score:", grid_search.best_score_)

best_rf = grid_search.best_estimator_
y_pred = best_rf.predict(X_test)

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, y_pred))
print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=3))

Fitting 5 folds for each of 36 candidates, totalling 180 fits
Beste parameters: {'max_depth': 16, 'min_samples_leaf': 20, 'n_estimators': 300}
Beste f1-score: 0.18700242565976327

Confusion Matrix:
[[66493   535]
 [  110  1062]]

Classification Report:
              precision    recall  f1-score   support

           0      0.998     0.992     0.995     67028
           1      0.665     0.906     0.767      1172

    accuracy                          0.991     68200
   macro avg      0.832     0.949     0.881     68200
weighted avg      0.993     0.991     0.991     68200



In [71]:
y_pred = rf.predict(X_test)

#Evaluatie 

print("Confusion Matrix:")
print(confusion_matrix(y_test, y_pred))

print("\nClassification Report:")
print(classification_report(y_test, y_pred, digits=3))

Confusion Matrix:
[[66989    39]
 [  831   341]]

Classification Report:
              precision    recall  f1-score   support

           0      0.988     0.999     0.994     67028
           1      0.897     0.291     0.439      1172

    accuracy                          0.987     68200
   macro avg      0.943     0.645     0.716     68200
weighted avg      0.986     0.987     0.984     68200



In [72]:
importance = pd.DataFrame({
    "feature": X_features,
    "importance": rf.feature_importances_
}).sort_values("importance", ascending=False)

print("\nTop Features:\n")
print(importance.head(28))


Top Features:

                                              feature  importance
15                            bufferlevel_roll_mean50    0.157022
13                            bufferlevel_roll_mean10    0.110758
14                            bufferlevel_roll_mean20    0.097733
0                                         bufferlevel    0.077356
26                            stangbuff_0_roll_mean20    0.062606
11                                   bufferlevel_lag2    0.057529
1                                    stangkwijt_timer    0.051703
10                                   bufferlevel_lag1    0.044167
27                            stangbuff_0_roll_mean50    0.038139
12                                   bufferlevel_lag3    0.033802
16                              stangkwijt_timer_lag1    0.028292
21                       stangkwijt_timer_roll_mean50    0.028187
2                                         stangbuff_0    0.026641
8                                       totaal_aantal    0.0

In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

# scores uit model
y_prob = rf.predict_proba(X_test)[:,1]

# PROBEER threshold = 0.7 (strenger → minder false positives)
threshold = 0.7
y_pred_thresh = (y_prob > threshold).astype(int)

print("THRESHOLD =", threshold)
print("\nConfusion matrix:")
print(confusion_matrix(y_test, y_pred_thresh))
print("\nClassification report:")
print(classification_report(y_test, y_pred_thresh, digits=3))


In [ ]:
print("Aantal features:", len(X_features))
print("Features:\n", X_features)
